In [4]:
from tensorflow.keras.datasets.mnist import load_data

(X_train,y_train) , (X_test,y_test) =load_data()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)


In [5]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K 

inp = Input(shape=(28,28,1))

x = Conv2D(filters=8 ,kernel_size=3, activation='relu')(inp)
x = MaxPooling2D(pool_size= (2,2))(x)

x =Flatten()(x)
x =Dense(units = 32,activation='relu')(x)
x = Dense(units =2)(x)

cnn = Model(inputs = inp , outputs = x )


img1 =Input(shape=(28,28,1))
img2 = Input(shape=(28,28,1))

f1 = cnn(img1)
f2 = cnn(img2)

d = K.sqrt(K.sum(K.square(f1-f2),axis =1 ,keepdims=True))

model = Model(inputs = (img1,img2),outputs = d)
model.summary()
cnn.summary()

def loss(y_true,y_pred):
    proba = K.exp(-K.square(y_pred))
    return -K.mean(y_true*K.log(proba)+(1-y_true)*K.log(1-proba))

def loss1(y_true,y_pred):
    return K.mean(y_true*K.square(y_pred)+(1-y_true)*K.square(K.maximum(1.0-y_pred,0)))

model.compile(optimizer='adam',loss = loss1)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
functional_1 (Functional)       (None, 2)            43442       input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Sub (TensorFlowOpLa [(None, 2)]          0           functional_1[0][0]    

Make all pairs or other strategies ; some innovation here

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt
def generator(X,y,k=8):
    unique_labels = np.unique(y)
    while True:
        X1 = []
        X2 = []
        y_batch = []
        for label in unique_labels :
            label_idx = np.where(y==label)[0]
            other_labels =set(unique_labels)-{label}
            for i in range(k):
                i1 = np.random.choice(label_idx)
                i2 = np.random.choice(label_idx)
                # i1 must be different from i2
                
                while i1 == i2 :
                    #i1 = np.random.choice(label_idx)
                    i2 = np.random.choice(label_idx)
                    
                # create positive example 
                X1.append(X[i1][:,:,None])
                X2.append(X[i2][:,:,None])
                y_batch.append(1.0)
                
                # create negative example
                i1 = np.random.choice(label_idx)
                my_label = np.random.choice(list(other_labels))
                i2 = np.random.choice(list(np.where(y == my_label)[0]))
                X1.append(X[i1][:,:,None])
                X2.append(X[i2][:,:,None])
                y_batch.append(0.0)
        
        yield [np.array(X1)/255.,np.array(X2)/255.],np.array(y_batch) 
            
                
                    
                    
                    